In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk 
from nltk.tokenize import word_tokenize
import emoji
from emoji.unicode_codes import UNICODE_EMOJI
from textblob import TextBlob

import sys
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def sentimentTextBlob(file):

    #loading the data into the dataframe
    data=pd.read_csv(file, encoding='utf-8')
    
    data['text']=data.astype(str).apply(' '.join, axis=1)
    data=pd.DataFrame(data['text'])
    

    #Removing the duplicate rows from text column and resetting index
    data=data.drop_duplicates(['text'],keep='first')
    data=data.reset_index(drop=True)
    data['Original Text']=data['text']

    #converting emoji into the text
    import emoji
    for i in range(len(data)):
        data.loc[i,'text'] = emoji.demojize(data.loc[i,'text'])


    #converting special character "’" to "'" for contraction
    for i in range(len(data)):
        data.loc[i,'text']=data.loc[i,'text'].replace("’","'")

    sys.path.insert(0, 'C:\\Users\\gautam\\Desktop\\Tweets_Notebook')
    from contractions_1 import CONTRACTION_MAP

    def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):

        contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                          flags=re.IGNORECASE|re.DOTALL)
        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match)\
                                    if contraction_mapping.get(match)\
                                    else contraction_mapping.get(match.lower())                       
            expanded_contraction = first_char+expanded_contraction[1:]
            return expanded_contraction

        expanded_text = contractions_pattern.sub(expand_match, text)
        expanded_text = re.sub("'", "", expanded_text)
        return expanded_text

    for i in range(len(data)):
        data.loc[i,'text'] = expand_contractions(str(data.loc[i,'text']))


    #data cleaning steps
    for i in range(len(data)):
        # Remove the word starting with @
        data.loc[i,'text'] = re.sub(r'@[A-Za-z0-9]+', ' ', str(data.loc[i,'text']))

        #Remove URL links
        data.loc[i,'text']  = re.sub('https?://[A-Za-z0-9./]+',' ',data.loc[i,'text'] ) 
        data.loc[i,'text']  = re.sub('http?://[A-Za-z0-9./]+',' ',data.loc[i,'text'] )


        # Converting to Lowercase
        data.loc[i,'text']  = data.loc[i,'text'] .lower()

        #Remove the new line characters
        data.loc[i,'text'] = re.sub(r"\t|\n|\r", " ", data.loc[i,'text'] , flags=re.I)

        #Remove punctuation
        data.loc[i,'text'] = re.sub(r"[,‘@\#-:'?\.$%_!()&;+”/…*•|“]", " ", data.loc[i,'text'] , flags=re.I)

        #Remove duble quotes
        data.loc[i,'text'] = re.sub(r'"', " ", data.loc[i,'text'] , flags=re.I)

        #Remove digits
        data.loc[i,'text'] = re.sub(r"\d", "", data.loc[i,'text'] )

        # remove all single characters
        data.loc[i,'text'] = re.sub(r'\s+[a-zA-Z]\s+', ' ', data.loc[i,'text'] )

        # Substituting multiple spaces with single space
        data.loc[i,'text']  = re.sub(r'\s+', ' ', data.loc[i,'text'] , flags=re.I)


    for i in range(len(data)):
        # removing rt from the data
        data.loc[i, 'text']=' '.join([x for x in data.loc[i,'text'].split() if x !='rt' and x !='nan'])  

         # remove all single characters
        data.loc[i,'text'] = re.sub(r'\s+[a-zA-Z]\s+', ' ', data.loc[i,'text'] )

        # Substituting multiple spaces with single space
        data.loc[i,'text']  = re.sub(r'\s+', ' ', data.loc[i,'text'] , flags=re.I)

    # remove remaining tokens that are not alphabetic
    for i in range(len(data)):
        data.loc[i, 'text']=' '.join([x for x in data.loc[i,'text'].split() if x.isalpha()])
    #     data.loc[i,'text'] = [word for word in data.loc[i,'text'] if word.isalpha()]



    stop_words = stopwords.words('english')
    stop_words.remove('no')
    stop_words.remove('not')
    for i in range(len(data)):
        data.loc[i,'text'] = ' '.join([word for word in data.loc[i,'text'].split() if not word in stop_words])


    # Lemmetization of words (it will change the word in the base form)
    lemmatizer = WordNetLemmatizer()
    for i in range(len(data)):
        data.loc[i,'text'] = ' '.join([lemmatizer.lemmatize(word,pos='a') for word in data.loc[i,'text'].split()])
        data.loc[i,'text'] = ' '.join([lemmatizer.lemmatize(word,pos='v') for word in data.loc[i,'text'].split()])
        data.loc[i,'text'] = ' '.join([lemmatizer.lemmatize(word,pos='n') for word in data.loc[i,'text'].split()])

    #Checking the words are english word or not, if not then remove it.
#     from nltk.corpus import words
#     for i in range(len(data)):
#         data.loc[i,'text'] = ' '.join([word for word in data.loc[i,'text'].split() if word in words.words()])


    for i in range(len(data)):
        # remove all single characters
        data.loc[i,'text'] = re.sub(r'\s+[a-zA-Z]\s+', ' ', data.loc[i,'text'] )

        # Substituting multiple spaces with single space
        data.loc[i,'text']  = re.sub(r'\s+', ' ', data.loc[i,'text'] , flags=re.I)


    for x in range(len(data)):
        #print(data_1.loc[x,'text'])
        data_sentiment=TextBlob(data.loc[x,'text'])
        #data.translate(from_lang='auto',to='en')
        data.loc[x,'Sentiment']=data_sentiment.sentiment.polarity

    data['Sentiment_rolled']= data['Sentiment'].apply(lambda x:  1 if x > 0.1 else (0 if x == 0  else -1))

    return data

In [2]:
file_1='D:\\Manisha\\Undergrad_Semester\\Semesters\\Fall2019\\capstone\\System.tweets_esports_olympics_Text.csv'

In [3]:
output=sentimentTextBlob(file_1)

In [4]:
output.head()

,text,Original Text,Sentiment,Sentiment_rolled
0,best player go head head whole world see start...,RT @IntelGaming: The best @RocketLeague player...,0.400000,1
1,trophy lequip special olympics catalunya sha p...,RT @btvesports: 🏆 L'equip Special Olympics Cat...,0.357143,1
2,best player go head head whole world see start...,RT @IntelGaming: The best @StreetFighter playe...,0.400000,1
3,,https://t.co/wBIbIBMLkc,0.000000,0
4,team international olympic committee intel wor...,The team-up of The International Olympic Commi...,0.000000,0


In [5]:
file_2='D:\\Manisha\\Undergrad_Semester\\Semesters\\Fall2019\\capstone\\data_sources\\Destination_Kent_State_Student_Evaluation_2017.csv'

In [6]:
output2=sentimentTextBlob(file_2)

In [7]:
output2.head(15)

,text,Original Text,Sentiment,Sentiment_rolled
0,open end response open end response open end r...,Open-Ended Response Open-Ended Response Open-E...,0.000000,0
1,graduate diploma give money,nan nan nan nan nan To graduate with a diploma...,0.000000,0
2,,nan nan nan nan nan nan nan,0.000000,0
3,campus tour lunch best,nan nan nan Campus tours Lunch To do my best. nan,1.000000,1
4,spend night dorm helpful involve campus activi...,nan nan nan nan Spending a night in the dorms ...,0.000000,0
5,no would tour downtown kent walk around downto...,nan nan nan No. It would be touring Downtown K...,0.450000,1
6,college different high school talk flash guide...,nan nan nan how college is different from high...,0.080000,-1
7,talk flashguides life like campus personal exp...,nan nan nan nan Talking to the flashguides abo...,0.266667,1
8,make sure set good amount time aside exam allo...,Make sure that they set a good amount of time ...,0.566667,1
9,buy book food live goal expectation,nan nan nan Buying books and when to do that. ...,0.136364,1
